# Data Exploration

## Import Libraries

In [1]:
from imageio import mimwrite, imwrite
from skimage.transform import resize
import matplotlib.pyplot as plt
import nibabel as nib
import pydicom as pdc
import numpy as np
import math
import sys
import os

In [2]:
%matplotlib inline

## Example of NIFTII format

In [3]:
nii_path = os.path.join('source', 'Task01_BrainTumour')
nii_path = os.path.join(nii_path, 'imagesTr')
data = nib.load(os.path.join(nii_path, 'BRATS_001.nii.gz'))

In [4]:
img = data.get_fdata()[:,:,:,3]
img = resize(img, (256, 256), mode='constant', clip=True, preserve_range=True)
img = (img - img.mean())/img.std()
img = 255 * (img - img.min())/(img.max() - img.min())

/Users/achauhan/anaconda3/envs/tensorflow/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [5]:
mimwrite(os.path.join('docs', 'nii_example.gif'), img.transpose((2,0,1)).astype('uint8')[10:140,:,:])

![nii_example](docs/nii_example.gif)

## Example of DICOM format

In [6]:
dicom_path = os.path.join(os.path.join('source', 'DIPG'), 'Stanford')
dicom_path = os.path.join(dicom_path, os.path.join('ST_DIPG_T2_Axial', 'no_roi'))

In [7]:
data_name = 'ST_DIPG_F_{:04d}-01-IM-0548'.format(60)
img = []
f = lambda x: x.startswith(data_name) and (len(x.split('-')) == 5 or (len(x.split('-')) == 6 and x.split('-')[-1]=='0001.dcm'))
for _, _, files in os.walk(dicom_path):
    files = sorted(filter(f, files))
    for filename in files:
        data = pdc.dcmread(os.path.join(dicom_path, filename))
        arr = data.pixel_array
        dim1 = max(arr.shape) - arr.shape[0]
        dim2 = max(arr.shape) - arr.shape[1]
        pad = np.pad(arr, ((math.ceil(dim1/2.0),math.floor(dim1/2.0)),
                           (math.ceil(dim2/2.0),math.floor(dim2/2.0))),
                     mode='constant', constant_values=0)
        img.append(resize(pad, (256, 256), mode='constant',
                          clip=True, preserve_range=True).tolist())

In [8]:
img = np.asarray(img)
img = (img - img.mean())/img.std()
img = 255 * (img - img.min())/(img.max() - img.min())

In [9]:
mimwrite(os.path.join('docs', 'dicom_example.gif'), np.rot90(img.astype('uint8'), axes=(2,1)))

![dicom_example](docs/dicom_example.gif)

## Example of NPZ format (from Stanford Healthy Brains)

In [10]:
npz_path = os.path.join('source', 'Normal')
arr = np.load(os.path.join(npz_path, '110M_67.npz'))['T2 ax']
dim1 = max(arr.shape[:2]) - arr.shape[0]
dim2 = max(arr.shape[:2]) - arr.shape[1]
pad = np.pad(arr, ((math.ceil(dim1/2.0),math.floor(dim1/2.0)),
                   (math.ceil(dim2/2.0),math.floor(dim2/2.0)), (0, 0)),
             mode='constant', constant_values=0)
img = resize(pad, (256, 256), mode='constant', clip=True, preserve_range=True)
img = img.transpose((2,0,1))

In [11]:
img = (img - img.mean())/img.std()
img = 255 * (img - img.min())/(img.max() - img.min())

In [12]:
mimwrite(os.path.join('docs', 'npz_example.gif'), np.rot90(img.astype('uint8'), axes=(2,1)))

![npz_example](docs/npz_example.gif)